In [78]:
# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.dates as dates
# import matplotlib.ticker as ticker
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import sched, time
import pandas_datareader as datareader
from pandas_datareader.data import Options
#ts = TimeSeries(key='5HZEUI5AFJB06BUK',output_format='pandas')
#key2 = '6ZAZOL7YF8VPXND7'
import matplotlib.pyplot as plt
import urllib.request as urlreq
from collections import OrderedDict
import statsmodels.formula.api as sm
%matplotlib inline

import plotly.plotly as py
import plotly.graph_objs as go

from pandas_datareader.data import Options
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *

from helpers import *

def greeks(options_df):
    data = options_df[['Symbol','Expiry','Strike', 'DTE', 'Type', 'IV', 'Vol','Open_Int', 
                       'Root', 'Underlying_Price','Last','Bid','Ask', 'Quote_Time']]
    data['Mid'] = (data['Ask'] - data['Bid'])/2 + data['Bid']
    
    year = 365
    strikes = data['Strike'].values
    time_to_expirations = data['DTE'].values
    underlying = data['Underlying_Price'].values[0]
    types = data['Type'].values

    # Make sure nothing thows up
    assert len(strikes) == len(time_to_expirations)

    sigmas = data['IV']
    deltas = []
    gammas = []
    thetas = []
    vegas = []
    for sigma, strike, time_to_expiration, flag in zip(sigmas, strikes, time_to_expirations, types):

        # Constants
        S = underlying
        K = strike
        t = time_to_expiration/float(year)
        r = 0.005 / 100
        q = 0 / 100

        try:
            delta = py_vollib.black_scholes_merton.greeks.analytical.delta(flag[0], S, K, t, r, sigma, q)
            deltas.append(delta)
        except:
            delta = 0.0
            deltas.append(delta)

        try:
            gamma = py_vollib.black_scholes_merton.greeks.analytical.gamma(flag[0], S, K, t, r, sigma, q)
            gammas.append(gamma)
        except:
            gamma = 0.0
            gammas.append(gamma)

        try:
            theta = py_vollib.black_scholes_merton.greeks.analytical.theta(flag[0], S, K, t, r, sigma, q)
            thetas.append(theta)
        except:
            theta = 0.0
            thetas.append(theta)

        try:
            vega = py_vollib.black_scholes_merton.greeks.analytical.vega(flag[0], S, K, t, r, sigma, q)
            vegas.append(vega)
        except:
            vega = 0.0
            vegas.append(vega)

    data['Delta'] = deltas
    data['Gamma'] = gammas
    data['Theta'] = thetas
    data['Vega'] = vegas
    data['Actual_Time'] = pd.to_datetime(data.Quote_Time) - dt.timedelta(hours = 4)
    data = data.set_index(data['Actual_Time'])
    data.index = data.index.map(lambda x: x.replace(second=0))
    return data#.reset_index()[data.columns]#data.dropna().reset_index()[data.columns]

## Aggregating Daily Data and Consolidating into one DataFrame

In [82]:
os.chdir('D:\Options Data\SPX')

start_time = time.time()

drop_thresh = 0.1
spike_thresh = 0.1
max_dte = 5

strike_lb = spx_options_df.Underlying_Price.min()*(1 - drop_thresh)
strike_ub = spx_options_df.Underlying_Price.max()*(1 + spike_thresh)

spx_options_hist = []

for file in os.listdir():
    spx_options_hist.append(pd.read_csv(file, index_col = 0))

spx_options_df = pd.concat(spx_options_hist, axis = 0).drop_duplicates()
spx_options_df = spx_options_df.set_index(pd.to_datetime(spx_options_df['Quote_Time']))
spx_options_df['DTE'] = (pd.to_datetime(spx_options_df['Expiry']) - pd.to_datetime(spx_options_df['Quote_Time'])).dt.days
spx_options_df = spx_options_df[(((spx_options_df.Strike >= strike_lb) & 
                                  (spx_options_df.Strike <= spx_options_df.Underlying_Price) &
                                  (spx_options_df.Type == 'put')) |
                                 ((spx_options_df.Strike <= strike_up) & 
                                 (spx_options_df.Strike >= spx_options_df.Underlying_Price) &
                                 (spx_options_df.Type == 'call'))) &
                                (spx_options_df.DTE <= max_dte)]

spx_options_df = greeks(spx_options_df)

print("--- %s seconds ---" % (time.time() - start_time))

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\py_vollib\ref_python\black_scholes_merton\__init__.py:87: RuntimeWarning:

divide by zero encountered in double_scalars

C:\Users\Fang\Anaconda3\lib\site-packages\py_vollib\black_scholes_merton\greeks\analytical.py:210: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\Fang\Anaconda3\lib\site-packages\py_vollib\black_scholes_merton\greeks\analytical.py:153: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\Fang\Anaconda3\lib\site-packages\py_vollib\ref_python\black_scholes_merton\__init__.py:86: RuntimeWarning:

invalid value encountered in sqrt

C:\Users\Fang\Anaconda

--- 54.899351358413696 seconds ---


In [92]:
options_filtered = spx_options[['Symbol','Expiry','DTE','Strike',
                                'Type','Last','Bid','Ask','Mid','Underlying_Price',
                                'Delta']].drop_duplicates()
puts = options_filtered[(options_filtered['Type'] == 'put') &
                        (options_filtered['Strike'] <= 2840) &
                        (options_filtered['Strike'] >= 2780)]
puts

Symbol               Expiry  DTE  Strike  \
Actual_Time                                                                  
2018-07-26 12:30:00  SPXW180727P02780000  2018-07-27 00:00:00    0  2780.0   
2018-07-26 12:30:00  SPXW180730P02780000  2018-07-30 00:00:00    3  2780.0   
2018-07-26 12:30:00  SPXW180731P02780000  2018-07-31 00:00:00    4  2780.0   
2018-07-26 12:30:00  SPXW180801P02780000  2018-08-01 00:00:00    5  2780.0   
2018-07-26 12:30:00  SPXW180727P02785000  2018-07-27 00:00:00    0  2785.0   
2018-07-26 12:30:00  SPXW180730P02785000  2018-07-30 00:00:00    3  2785.0   
2018-07-26 12:30:00  SPXW180731P02785000  2018-07-31 00:00:00    4  2785.0   
2018-07-26 12:30:00  SPXW180801P02785000  2018-08-01 00:00:00    5  2785.0   
2018-07-26 12:30:00  SPXW180727P02790000  2018-07-27 00:00:00    0  2790.0   
2018-07-26 12:30:00  SPXW180730P02790000  2018-07-30 00:00:00    3  2790.0   
2018-07-26 12:30:00  SPXW180801P02790000  2018-08-01 00:00:00    5  2790.0   
2018-07-26 12:30:00  SPXW180727P02795000  2018-07-27 00:00:00    0  2795.0   
2018-07-26 12:30:00  SPXW180730P02795000  2018-07-30 00:00:00    3  2795.0   
2018-07-26 12:30:00  SPXW180731P02795000  2018-07-31 00:00:00    4  2795.0   
2018-07-26 12:30:00  SPXW180801P02795000  2018-08-01 00:00:00    5  2795.0   
2018-07-26 12:30:00  SPXW180727P02800000  2018-07-27 00:00:00    0  2800.0   
2018-07-26 12:30:00  SPXW180730P02800000  2018-07-30 00:00:00    3  2800.0   
2018-07-26 12:30:00  SPXW180801P02800000  2018-08-01 00:00:00    5  2800.0   
2018-07-26 12:30:00  SPXW180727P02805000  2018-07-27 00:00:00    0  2805.0   
2018-07-26 12:30:00  SPXW180730P02805000  2018-07-30 00:00:00    3  2805.0   
2018-07-26 12:30:00  SPXW180731P02805000  2018-07-31 00:00:00    4  2805.0   
2018-07-26 12:30:00  SPXW180801P02805000  2018-08-01 00:00:00    5  2805.0   
2018-07-26 12:30:00  SPXW180727P02810000  2018-07-27 00:00:00    0  2810.0   
2018-07-26 12:30:00  SPXW180730P02810000  2018-07-30 00:00:00    3  2810.0   
2018-07-26 12:30:00  SPXW180801P02810000  2018-08-01 00:00:00    5  2810.0   
2018-07-26 12:30:00  SPXW180727P02815000  2018-07-27 00:00:00    0  2815.0   
2018-07-26 12:30:00  SPXW180730P02815000  2018-07-30 00:00:00    3  2815.0   
2018-07-26 12:30:00  SPXW180731P02815000  2018-07-31 00:00:00    4  2815.0   
2018-07-26 12:30:00  SPXW180801P02815000  2018-08-01 00:00:00    5  2815.0   
2018-07-26 12:30:00  SPXW180727P02820000  2018-07-27 00:00:00    0  2820.0   
...                                  ...                  ...  ...     ...   
2018-08-03 16:10:00  SPXW180803P02835000  2018-08-03 00:00:00   -1  2835.0   
2018-08-03 16:10:00  SPXW180806P02835000  2018-08-06 00:00:00    2  2835.0   
2018-08-03 16:10:00  SPXW180808P02835000  2018-08-08 00:00:00    4  2835.0   
2018-08-03 16:10:00  SPXW180803P02840000  2018-08-03 00:00:00   -1  2840.0   
2018-08-03 16:10:00  SPXW180806P02840000  2018-08-06 00:00:00    2  2840.0   
2018-08-03 16:10:00  SPXW180808P02840000  2018-08-08 00:00:00    4  2840.0   
2018-08-03 16:15:00  SPXW180806P02780000  2018-08-06 00:00:00    2  2780.0   
2018-08-03 16:15:00  SPXW180808P02780000  2018-08-08 00:00:00    4  2780.0   
2018-08-03 16:15:00  SPXW180808P02785000  2018-08-08 00:00:00    4  2785.0   
2018-08-03 16:15:00  SPXW180803P02790000  2018-08-03 00:00:00   -1  2790.0   
2018-08-03 16:15:00  SPXW180806P02790000  2018-08-06 00:00:00    2  2790.0   
2018-08-03 16:15:00  SPXW180808P02790000  2018-08-08 00:00:00    4  2790.0   
2018-08-03 16:15:00  SPXW180808P02795000  2018-08-08 00:00:00    4  2795.0   
2018-08-03 16:15:00  SPXW180806P02800000  2018-08-06 00:00:00    2  2800.0   
2018-08-03 16:15:00  SPXW180808P02800000  2018-08-08 00:00:00    4  2800.0   
2018-08-03 16:15:00  SPXW180806P02805000  2018-08-06 00:00:00    2  2805.0   
2018-08-03 16:15:00  SPXW180808P02805000  2018-08-08 00:00:00    4  2805.0   
2018-08-03 16:15:00  SPXW180806P02810000  2018-08-06 00:00:00    2  2810.0   
2018-08-03 16:15:00  SPXW180808P02810000  2018-08-08 00:

In [98]:
os.chdir('D:\Options Data\SPX Intraday')

spx_intraday = []

for file in os.listdir():
    spx_intraday.append(pd.read_csv(file, index_col = 0))

spx_intraday = pd.concat(spx_intraday, axis = 0).drop_duplicates()
spx_intraday = spx_intraday.set_index(pd.to_datetime(spx_intraday.index))
spx_intraday['Fast MA'] = spx_intraday['Last'].rolling(5).mean()
spx_intraday['Slow MA'] = spx_intraday['Last'].rolling(20).mean()

In [104]:
backtesting = puts.join(spx_intraday, rsuffix = '_spx').dropna()
backtesting.to_csv('test.csv')